In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

if "../" not in sys.path:
    sys.path.append("../")
    print("[sys.path]:", sys.path)

[sys.path]: ['/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/nadir/MoLiNER/.venv/lib/python3.10/site-packages', '../']


In [3]:
import torch

In [4]:
from src.model.moliner import MoLiNER

from src.model.modules.motion_frames_encoders import TMRMotionFramesEncoder
from src.model.modules.prompts_tokens_encoders import CLIPPromptsTokensEncoder
from src.model.modules.prompt_representation_layers import MLPPromptRepresentationLayer
from src.model.modules.span_representation_layers import EndpointsSpanRepresentationLayer
from src.model.modules.scorers import ProductPairScorer
from src.model.modules.optimizers import AdamOptimizer
from src.model.modules.decoders import GreedyDecoder
from src.model.modules.spans_generators.static import StaticSpansGenerator

from src.model.losses import FocalLoss

/home/nadir/MoLiNER/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
model = MoLiNER(
    motion_frames_encoder=TMRMotionFramesEncoder(frozen=False, pretrained=False),
    prompts_tokens_encoder=CLIPPromptsTokensEncoder(frozen=True),
    prompt_representation_layer=MLPPromptRepresentationLayer(input_dim=512, representation_dim=256, dropout=0.4),
    span_representation_layer=EndpointsSpanRepresentationLayer(motion_embed_dim=256, representation_dim=256, dropout=0.4),
    scorer=ProductPairScorer(),
    optimizer=AdamOptimizer({
        "scratch": 5e-5,
        "pretrained": 1e-5
    }),
    decoder=GreedyDecoder(),
    spans_generator=StaticSpansGenerator(max_width=10, min_width=1, padding_value=-1),
    loss=FocalLoss(
        alpha=0.25,
        gamma=2.0,
        # NOTE: 'none', 'mean', 'sum'
        reduction="sum",
        label_smoothing=0.0,
        # NOTE: 'labels', 'global', 'span'
        negatives_type="labels",
        negatives_probability=1.0,
        ignore_index=-100,
        threshold=0.75,
    )
).to("cuda:1")

In [6]:
print(f"Allocated: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
print(f"Reserved: {torch.cuda.memory_reserved() / 1024**2:.2f} MB")

Allocated: 0.00 MB
Reserved: 0.00 MB


In [7]:
model = model.eval()

In [8]:
compiled = torch.compile(model, fullgraph=True)

In [9]:
import os

os.environ["TORCHDYNAMO_VERBOSE"] = "1"

In [10]:
from src.types import RawBatch, ProcessedBatch

raw_batch = RawBatch.create_random()
processed_batch = ProcessedBatch.from_raw_batch(raw_batch, encoder=model.prompts_tokens_encoder).to("cuda:1")

In [11]:
normal_model_output = model.forward(processed_batch)
compiled_model_output = compiled.forward(processed_batch)

Unsupported: torch.* op returned non-Tensor bool call_function <built-in method _nested_tensor_from_mask_left_aligned of type object at 0x77b9c6da0f40>

from user code:
   File "/home/nadir/MoLiNER/tests/../src/model/moliner.py", line 98, in forward
    motion_frames_embeddings = self.motion_frames_encoder.forward(
  File "/home/nadir/MoLiNER/tests/../src/model/modules/motion_frames_encoders/tmr.py", line 82, in forward
    cls_token, full_sequence = self.tmr_encoder.forward(
  File "/home/nadir/MoLiNER/tests/../src/model/helpers/actor.py", line 113, in forward
    final = self.seqTransEncoder(xseq, src_key_padding_mask=(aug_mask == 0).to(torch.bool))
  File "/home/nadir/MoLiNER/.venv/lib/python3.10/site-packages/torch/nn/modules/transformer.py", line 457, in forward
    ) and not torch._nested_tensor_from_mask_left_aligned(

Set TORCHDYNAMO_VERBOSE=1 for the internal stack trace (please do this especially if you're reporting a bug to PyTorch). For even more developer context, set TORC

In [60]:
index = 0, 0, 3

print(normal_model_output.similarity_matrix[index])
print(compiled_model_output.similarity_matrix[index])

tensor(0.5017, device='cuda:1', grad_fn=<SelectBackward0>)
tensor(0.5017, device='cuda:1', grad_fn=<SelectBackward0>)


In [43]:
torch.allclose(normal_model_output.similarity_matrix, compiled_model_output.similarity_matrix, atol=1e-6)

False